# Systematic Investment Final Project
### 2019 NYU Stern Spring Semester

#### Principal Investigators: Ruijie (Cherry) Cai (rc3470),  Zhiyu Chen (zc987) <br> Email: ruijiu.cai@stern.nyu.edu, zhiyu.chen@stern.nyu.edu


# Data Cleaning

**Overview:** The data behind our project comes from [“Tushare”](https://tushare.pro/), which is a Chinese financial data platforms for stocks, funds, futures, bonds, foreign exchange, industry big data, and block-chain data. As mentioned above, their [historical daily stock price](https://tushare.pro/document/2?doc_id=27) provides access to all the data we need.

**Important Variables:** The key series that we must retrieve are as follows which are defined as:

- **Close Price**
The final price at which a stock is traded on a given trading day which represents the most up-to-date valuation of a security until trading commences again on the next trading day.

- **Trading Volumn**
In capital markets, it measures the the amount of a security that was traded during a given period of time. In the context of a single stock trading on a stock exchange, it is commonly reported as the number of shares that changed hands during a given day. 

**Access**: We will install and launch the [Tushare library](https://tushare.pro/document/1?doc_id=7) to download and access the data. Below We demonstrate that We have the ability to access the data.

In [1]:
import tushare as ts                   # A Chinese financial data platform. Install this model to get access to the stock price
import pandas as pd                    # data package
import matplotlib.pyplot as plt        # graphics module  
import datetime as dt                  # date and time module
import numpy as np                     # foundation for pandas 

In [2]:
#in order to get access to the financial data in Tushare, we need to apply for an account and get this token

ts.set_token("2b8517650616c498e2621ed18ea7217731b6e09879a8fa917156f0b2")

pro = ts.pro_api()

data = pro.stock_basic(exchange='', list_status='L', fields='ts_code,symbol,name,area,industry,list_date').set_index("list_date")

## 1. Grabing the Historical Daily Stock Close Price and Trading Volume for PetroChina & Sinopec

In [3]:
#now we need to grab the historical daily index for petrochina and sinopec stocks from March 22 2010 till 
#april 19 2019. To be more specific, the timeslot covers the last 10 years plus a 21 days for us to calculate the 
#return and the 20-day volatility

petrochina = pro.query('daily', ts_code='601857.SH', start_date='20100322', end_date='20190419')
petrochina = petrochina.set_index(pd.to_datetime(petrochina["trade_date"], format="%Y-%m-%d")) #set the index as the trading date
petrochina = petrochina[["ts_code","close","vol"]] # Grab only the data we want
petrochina = petrochina.iloc[::-1,:] # Transpose timeline

sinopec = pro.query('daily', ts_code='600028.SH', start_date='20100322', end_date='20190419')
sinopec = petrochina.set_index(pd.to_datetime(sinopec["trade_date"], format="%Y-%m-%d")) #set the index as the trading date
sinopec = sinopec[["ts_code","close","vol"]] # Grab only the data we want
sinopec = sinopec.iloc[::-1,:] # Transpose timeline

In [4]:
petrochina.head()

,ts_code,close,vol
trade_date,,,
2010-03-22,601857.SH,12.85,154858.22
2010-03-23,601857.SH,12.81,142662.43
2010-03-24,601857.SH,12.77,151031.25
2010-03-25,601857.SH,12.65,149957.53
2010-03-26,601857.SH,12.77,194796.52


In [5]:
sinopec.head()

,ts_code,close,vol
trade_date,,,
2010-03-22,601857.SH,7.66,955482.80
2010-03-23,601857.SH,7.67,748744.96
2010-03-24,601857.SH,7.68,996358.73
2010-03-25,601857.SH,7.73,1248653.56
2010-03-26,601857.SH,7.61,956048.18


## 2. Grabing the Historical Daily Stock Close Price and Trading volume for for All Listed Stocks in the Oil Industry in China

In [6]:
# This is the dataframe access to the ts_code, name, industry of the stock
data.head()

,ts_code,symbol,name,area,industry
list_date,,,,,
19910403,000001.SZ,000001,平安银行,深圳,银行
19910129,000002.SZ,000002,万科A,深圳,全国地产
19910114,000004.SZ,000004,国农科技,深圳,生物制药
19901210,000005.SZ,000005,世纪星源,深圳,环境保护
19920427,000006.SZ,000006,深振业A,深圳,区域地产


In [7]:
data.industry.unique() #To get name of the industries of the stock
# We found out that there are three categories that related to the oil industry("石油加工", "石油贸易", "石油开采")

array(['银行', '全国地产', '生物制药', '环境保护', '区域地产', '酒店餐饮', '运输设备', '综合类',
       '建筑施工', '玻璃', '家用电器', '文教休闲', '装修装饰', '其他商业', '元器件', '电脑设备',
       '其他建材', '汽车服务', '火力发电', '医药商业', '汽车配件', '广告包装', '轻工机械', '新型电力',
       '房产服务', '纺织', '饲料', '电气设备', '石油加工', '铅锌', '农业综合', '批发业', '通信设备',
       '旅游景点', '港口', '机场', '石油贸易', '空运', '医疗保健', '商贸代理', '化学制药', '农药化肥',
       '影视音像', '工程机械', '证券', '化纤', '水泥', '专用机械', '供气供热', '机床制造', '多元金融',
       '百货', '中成药', '路桥', '造纸', '食品', '化工原料', '矿物制品', '水运', '日用化工',
       '机械基件', '汽车整车', '煤炭开采', '软件服务', '铁路', '染料涂料', '白酒', '林业', '水务',
       '水力发电', '互联网', '旅游服务', '铝', '保险', '园区开发', '小金属', '铜', '普钢', '纺织机械',
       '航空', '特种钢', '种植业', '出版业', '焦炭加工', '啤酒', '公路', '超市连锁', '钢加工', '渔业',
       '农用机械', '软饮料', '化工机械', '塑料', '红黄药酒', '橡胶', '家居用品', '摩托车', '电器仪表',
       '服饰', '仓储物流', '电器连锁', '半导体', '电信运营', '黄金', '石油开采', '乳制品', '商品城',
       '公共交通', '船舶', '陶瓷'], dtype=object)

In [8]:
# Now we want to grab the histroical daily index for all the stock in the oil industry in China
# First we need to know the ts_code of them and then set the index as their ts_code
oil_processing = pd.DataFrame(data.loc[data["industry"] == "石油加工"]).set_index("ts_code")
oil_trading = pd.DataFrame(data.loc[data["industry"] == "石油贸易"]).set_index("ts_code")
oil_deplore = pd.DataFrame(data.loc[data["industry"] == "石油开采"]).set_index("ts_code")

# In order to Processing the for loop in the following we create a list for each of three category
oil_processing_list = oil_processing.index.tolist()
oil_trading_list = oil_trading.index.tolist()
oil_deplore_list = oil_deplore.index.tolist()
company_list = oil_processing_list + oil_trading_list + oil_deplore_list
# This is the list for the whole industry

## First we are going to grab the close price for all listed stock

In [9]:
# to use the join function in the for loop, first we need to create an dataframe. Here we import the first stock in 
# the listed stocks

company_df = pro.query('daily', ts_code="000059.SZ", start_date='20130419', end_date='20190420') 

company_df = company_df.set_index(company_df["trade_date"])
# We changed the index into the trading date

company_df = pd.DataFrame(company_df["close"])
# We only grab the column for the close price

company_df = company_df.rename(columns = {"close":"000059.SZ"})
# We change the name of the column into the ts_code of the stock

# This is the for loop to join the other stoclk into this dataframe
for item in company_list:
    
    test = pro.query('daily', ts_code=item, start_date='201304020', end_date='20190420') 
    
    test = test.set_index(test["trade_date"])
    
    test = pd.DataFrame(test["close"])
    
    test = test.rename(columns = {"close":item})
    
    company_df = company_df.join(test, lsuffix = "trade_date")

company_df = company_df[company_list]

company_df = company_df.drop(columns=['002778.SZ', '603798.SH','603003.SH','002828.SZ','603619.SH','603727.SH'])
# We dropped those stocks that issued after the selected date

company_df = company_df.iloc[::-1,:]
# We reconstruct the index based on the timeline from past to present

company_df.head()

,000059.SZ,000637.SZ,000819.SZ,002377.SZ,600028.SH,600339.SH,600688.SH,000096.SZ,000159.SZ,000554.SZ,...,300084.SZ,300157.SZ,300164.SZ,300191.SZ,600256.SH,600583.SH,600759.SH,600871.SH,601808.SH,601857.SH
trade_date,,,,,,,,,,,,,,,,,,,,,
20130419,5.90,4.90,13.96,6.53,6.94,4.54,5.86,4.55,6.27,4.51,...,11.72,NaN,NaN,15.51,19.60,6.46,4.89,6.09,15.60,8.56
20130422,5.84,4.81,14.43,6.65,6.90,4.48,5.79,4.63,6.22,4.48,...,11.86,NaN,NaN,16.38,19.41,6.43,4.85,6.00,15.63,8.57
20130423,5.53,4.61,13.58,6.39,6.63,4.32,5.60,4.41,6.04,4.38,...,11.36,NaN,NaN,16.53,18.90,6.22,4.66,5.76,15.15,8.47
20130424,5.57,4.80,13.98,6.42,6.74,4.35,5.72,4.49,6.09,4.42,...,11.59,NaN,NaN,17.15,19.04,6.29,4.68,5.85,15.44,8.51
20130425,5.47,4.69,14.72,6.22,6.78,4.22,5.66,4.36,6.00,4.34,...,11.07,NaN,NaN,16.63,18.84,6.24,4.49,5.77,15.65,8.52


## Second we grab the trading volumn for all listed stock

In [10]:
# to use the join function in the for loop, first we need to create an dataframe. Here we import the first stock in 
# the listed stocks

company_df1 = pro.query('daily', ts_code="000059.SZ", start_date='20130419', end_date='20190420') 

company_df1 = company_df1.set_index(company_df1["trade_date"])
# We changed the index into the trading date

company_df1 = pd.DataFrame(company_df1["vol"])
# We only grab the column for the trading colume

company_df1 = company_df1.rename(columns = {"vol":"000059.SZ"})
# We change the name of the column into the ts_code of the stock

# This is the for loop to join the other stoclk into this dataframe
for item in company_list:
    
    test = pro.query('daily', ts_code=item, start_date='201304020', end_date='20190420') 
    
    test = test.set_index(test["trade_date"])
    
    test = pd.DataFrame(test["vol"])
    
    test = test.rename(columns = {"vol":item})
    
    company_df1 = company_df1.join(test, lsuffix = "trade_date")

company_df1 = company_df1[company_list]

company_df1 = company_df1.drop(columns=['002778.SZ', '603798.SH','603003.SH','002828.SZ','603619.SH','603727.SH'])
# We dropped those stocks that issued after the selected date

company_df1 = company_df1.iloc[::-1,:]
# We reconstruct the index based on the timeline from past to present

company_df1.head()

,000059.SZ,000637.SZ,000819.SZ,002377.SZ,600028.SH,600339.SH,600688.SH,000096.SZ,000159.SZ,000554.SZ,...,300084.SZ,300157.SZ,300164.SZ,300191.SZ,600256.SH,600583.SH,600759.SH,600871.SH,601808.SH,601857.SH
trade_date,,,,,,,,,,,,,,,,,,,,,
20130419,102158.66,110351.90,46020.39,10794.58,389251.52,85424.26,33065.36,61900.86,26730.36,28394.12,...,27617.62,NaN,NaN,39425.56,222142.16,240499.68,228921.88,12269.04,63000.25,226111.14
20130422,96787.14,68669.97,64829.00,13572.70,315942.30,43604.45,50563.89,61289.76,16602.50,37347.76,...,38350.27,NaN,NaN,61374.30,180590.28,185334.18,151802.03,23231.91,40760.58,132373.22
20130423,115965.57,72691.37,50702.05,11133.00,554432.52,42008.65,86455.92,44177.19,16186.96,27548.46,...,25033.74,NaN,NaN,55074.26,183059.35,142637.59,154218.07,21783.97,49868.68,143919.74
20130424,92844.23,75937.86,39240.88,12448.96,283744.11,32965.67,50738.99,20540.06,16804.67,46551.69,...,18617.68,NaN,NaN,63355.57,185032.84,168353.19,72445.30,33093.69,45569.49,129595.15
20130425,72263.80,49174.76,88876.19,10997.30,281228.15,36527.50,41531.37,17880.79,14846.91,34103.58,...,28505.22,NaN,NaN,42645.88,165698.93,147413.47,124766.05,16693.50,61880.17,132103.82


## To sum up for the Data Cleaning part

We got all the data we need:(name of the dataframe is in the bracket)
1. Daily Close price and trading volumn for petrochina (petrochina)
2. Daily Close price and trading volumn for sinopec (sinopec)
3. Daily Close price for all listed stock in Oil industry (company_df)
4. Daily Trading volume of the stock for all listed stock in Oil industry (company_df1)

In [11]:
# Now we want to download the excel file 
petrochina.to_excel("petrochina.xlsx")
sinopec.to_excel("sinopec.xlsx")
company_df.to_excel("all_stock_close.xlsx")
company_df1.to_excel("all_stock_volume.xlsx")